In [1]:
import sys
sys.path.append("/home/joishi/hg-projects/eigentools")

import numpy as np
from dedalus import public as de
from eigentools import Eigenproblem

# Taylor-Couette Linear Eigenvalue Benchmarking

In order to test our code, particularly the linear terms, we compare against the table provided in Marcus (1984).

## Comparing Marcus 1984 with Chandrashekar 1961


In Marcus 1984, the control parameter is the Reynolds number with:

\begin{align}
\mathrm{Re} = \frac{\Omega_{1} R_1 d}{\nu}
\end{align}
with $d = b-a$.

Meanwhile, in Chandrashekar the control parameter is the Taylor number (eqn 314):
\begin{align}
    \mathrm{Ta} = \frac{64}{9}\left(\frac{\Omega_{1} R_1^2}{\nu}\right)^2(1-\mu)(1-4\mu)
\end{align}
with $\mu = \Omega_2/\Omega_1$, $\eta = R_1/R_2$.  We can tie to Table 35, where $\eta = 0.5$ and where Marcus is pulling his comparison values from the cases with $\kappa=0.4$ and $\kappa=0.6$.


Converting Chandrashekar's $Ta$ to $Re$ we have
\begin{align}
    d = R_1 \frac{1-\eta}{\eta}
\end{align}
or
\begin{align}
    R_1 = d \frac{\eta}{1-\eta}
\end{align}
and
\begin{align}
    \mathrm{Ta} = \frac{64}{9}\left(\frac{\Omega_{1} R_1 d}{\nu}\frac{\eta}{1-\eta}\right)^2(1-\mu)(1-4\mu)
\end{align}
and
\begin{align}
    \mathrm{Ta} = \frac{64}{9}\left(\mathrm{Re}\frac{\eta}{1-\eta}\right)^2(1-\mu)(1-4\mu)
\end{align}
or
\begin{align}
\mathrm{Re}^2 = \mathrm{Ta}\frac{9}{64}\left(\frac{1-\eta}{\eta}\right)^2\frac{1}{(1-\mu)(1-4\mu)}
\end{align}

Chandra's two cases that Marcus claims to reproduce are at ($\eta = 1/2$):
\begin{align}
    \kappa = 0.4 \quad & \alpha \approx 0.117647 & \mathrm{Ta} \approx 1.954 \times 10^4 \\
    \kappa = 0.6 \quad & \alpha \approx 0.166667 & \mathrm{Ta} \approx 2.264 \times 10^4
\end{align}
Note that these are locally the only directly calcuated values in that table (values in table 35 that have daggers are interpolated).
    

In [2]:
η = 1/2
def Re(η,μ,Ta):
    return np.sqrt(9/64*((1-η)/η)**2*1/((1-μ)*(1-4*μ))*Ta)

The $\mathrm{Re}$ for a code normalized with Marcus's non-dimensionalization should be

In [3]:
# Chandra kappa=0.4
μ = 0.11765
Ta = 2.264e4
print("Re = {}".format(Re(η,μ,Ta)))

Re = 82.55760118425344


In [4]:
# Chandra kappa=0.6
μ = 0.16667
Ta = 1.954e4
print("Re = {}".format(Re(η,μ,Ta)))

Re = 99.46135063967873


though these are *not* the values reported for $\mathrm{Re}$. All other parameters are correct.

Krueger, Gross and DiPrima (1966)
-------------------------------

Marcus also compares to Krueger, Gross and DiPrima (1966).  Marcus appears to be pulling values from their Table 2, with $\eta=0.95$.


Let's convert their Taylor numbers into the appropriate Reynolds numbers.  Their $\mathrm{Ta}$ is in equations 5:
\begin{align}
    \mathrm{Ta} = - \frac{4 A \Omega_1 d^4}{\nu^2}
\end{align}
and with
\begin{align}
    A = -\Omega_1 (1-\mu)/(2\delta) \\
    \delta = \frac{d}{R_1} = \frac{1-\eta}{\eta}
\end{align}
we get
\begin{align}
    \mathrm{Ta} = \frac{4 \Omega_1^2 d^4}{\nu^2} \frac{1}{2}(1-\mu)\frac{\eta}{1-\eta}
\end{align}
and with
\begin{align}
    d^2 = R_1^2 \left(\frac{1-\eta}{\eta}\right)^2
\end{align}
we can write
\begin{align}
    \mathrm{Ta} = 2 \left(\frac{\Omega_1 R_1 d}{\nu}\right)^2 (1-\mu)\frac{(1-\eta)}{\eta}
\end{align}
or
\begin{align}
\mathrm{Re}^2 = \frac{1}{2}\frac{\eta}{1-\eta}\frac{1}{1-\mu}\mathrm{Ta}
\end{align}

The conversion of their reported frequencies $\sigma_r$ to the timescales of Marcus come from:
\begin{align}
    \tau_M = \frac{d}{\Omega_1 R_1} \\
    \tau_K = \frac{d^2}{\nu}
\end{align}  
This leads to a ratio of timescales:
\begin{align}
    \frac{\tau_K}{\tau_M} = \frac{\Omega_1 R_1 d}{\nu} = \mathrm{Re}
\end{align}

In [5]:
def Re_KGdP(η,μ,Ta):
    return np.sqrt(Ta*1/2*(η/(1-η))*1/(1-μ))

`KGdP_Ta` gives the input parameters. It is a dictionary of dictionaries holding all parameters as a function of $\mu$: $\mathrm{Ta}$, $\sigma$, $\alpha$, and $m$.

In the cell below, we take each of these parameter sets, calculuate the $\mathrm{Re}$ from $\mathrm{Ta}$, and rename all dictionary keys to match those parameters pre-defined in our eigenvalue solver. This is done because they match an already pre-existing initial value problem code. The output is `KGdP_Re`, which contains parameters ready made for our solver.

In [22]:
η = 0.95
KGdP_Ta = {0:{'Ta':3509.9, 'σ':0     , 'α': 3.128, 'm': 0}, 
        -0.8:{'Ta':13730,  'σ':15.106, 'α': 3.561, 'm': 3}, 
          -1:{'Ta':20072,  'σ':23.358, 'α': 3.680, 'm': 4},
       -1.25:{'Ta':30632,  'σ':33.102, 'α': 3.774, 'm': 5},
        -1.5:{'Ta':45307,  'σ':43.616, 'α': 4.002, 'm': 6},
       -1.75:{'Ta':65411,  'σ':51.537, 'α': 3.986, 'm': 6}, 
        -2.0:{'Ta':91298,  'σ':64.147, 'α': 4.483, 'm': 7}
       }
KGdP_Re = {μ: {} for μ in KGdP_Ta}
for μ in KGdP_Ta:
    Ta = KGdP_Ta[μ]['Ta']
    σ = KGdP_Ta[μ]['σ']
    Re = Re_KGdP(η,μ,Ta)
    KGdP_Re[μ]['nu'] = 1/Re
    KGdP_Re[μ]['mu'] = μ
    KGdP_Re[μ]['m'] = KGdP_Ta[μ]['m']
    KGdP_Re[μ]['eta'] = η
    KGdP_Re[μ]['kz'] = KGdP_Ta[μ]['α']

    print("μ = {:5.2f}:    Re = {} and  σ = {}".format(μ, Re, σ*KGdP_Re[μ]['nu']))

μ =  0.00:    Re = 182.60353227689754 and  σ = 0.0
μ = -0.80:    Re = 269.19117535478165 and  σ = 0.056116252622661136
μ = -1.00:    Re = 308.77499898793604 and  σ = 0.07564731625474835
μ = -1.25:    Re = 359.6319105851301 and  σ = 0.09204411239854166
μ = -1.50:    Re = 414.9296325884665 and  σ = 0.10511661875752078
μ = -1.75:    Re = 475.3580468733779 and  σ = 0.10841722431960434
μ = -2.00:    Re = 537.6898114464632 and  σ = 0.11930112610360855


## Check Growth Rates

Here, we check our results against all data reported in Marcus 1984a, correcting for $\mathrm{Re}$ as above for KGD66.

The next two cells contain the eigenvalue solver; below `nr` is the resolution. Other parameters are just the first row of the data table; they are only necessary to initialize the solver.

In [11]:
nr = 32

# default parameters from KDG66 Table 2, row 1
Re1 = 182.60353227689754
eta = 0.95
mu = 0.
alpha = 3.128
m = 0

In [12]:
"""
delta = R2 - R1
mu = Omega2/Omega1
eta = R1/R2

scale [L] = delta
scale [T] = delta/(R1 Omega1)
scale [V] = R1 Omega1
"""
#derived parameters
R1 = eta/(1. - eta)
R2 = 1./(1-eta)
nu = 1./Re1

variables = ['u','ur','v','vr','w','wr','p']

#domain
r_basis = de.Chebyshev('r', nr, interval=[R1, R2])

bases = [r_basis]
domain = de.Domain(bases) 

#problem
problem = de.EVP(domain, eigenvalue='sigma', variables=variables)

#params into equations
problem.parameters['eta']=eta
problem.parameters['mu']=mu
problem.parameters['nu']=nu
problem.parameters['kz'] = alpha
problem.parameters['m'] = m

#Substitutions

"""
this implements the cylindrical del operators. 
NB: ASSUMES THE EQUATION SET IS PREMULTIPLIED BY A POWER OF r (SEE BELOW)!!!

Lap_s --> scalar laplacian
Lap_r --> r component of vector laplacian
Lap_t --> theta component of vector laplacian
Lap_z --> z component of vector laplacian

"""

problem.substitutions['A'] = '(1/eta - 1.)*(mu-eta**2)/(1-eta**2)'
problem.substitutions['B'] = 'eta*(1-mu)/((1-eta)*(1-eta**2))'

problem.substitutions['v0'] = 'A*r + B/r'       #background profile? forcing instead of forcing the boundaries
problem.substitutions['dv0dr'] = 'A - B/(r*r)'  #d/dr of background forcing

problem.substitutions['dtheta(f)'] = '1j*m*f'
problem.substitutions['dz(f)'] = '1j*kz*f'
problem.substitutions['dt(f)'] = 'sigma*f'

# assume pre-multiplication by r*r
problem.substitutions['Lap_s(f, f_r)'] = "r*r*dr(f_r) + r*f_r + dtheta(dtheta(f)) + r*r*dz(dz(f))"
problem.substitutions['Lap_r'] = "Lap_s(u, ur) - u - 2*dtheta(v)"
problem.substitutions['Lap_t'] = "Lap_s(v, vr) - v + 2*dtheta(u)"
problem.substitutions['Lap_z'] = "Lap_s(w, wr)"

# momentum equations
problem.add_equation("r*r*dt(u) - nu*Lap_r - 2*r*v0*v + r*v0*dtheta(u) + r*r*dr(p) = 0")
problem.add_equation("r*r*dt(v) - nu*Lap_t + r*r*dv0dr*u + r*v0*u + r*v0*dtheta(v) + r*dtheta(p)  = 0")
problem.add_equation("r*r*dt(w) - nu*Lap_z + r*r*dz(p) + r*v0*dtheta(w) = 0.")

#continuity
problem.add_equation("r*ur + u + dtheta(v) + r*dz(w) = 0")

#Auxillilary equations
problem.add_equation("ur - dr(u) = 0")
problem.add_equation("vr - dr(v) = 0")
problem.add_equation("wr - dr(w) = 0")

#Boundary Conditions
problem.add_bc("left(u) = 0")
problem.add_bc("right(u) = 0")
problem.add_bc("left(v) = 0")
problem.add_bc("right(v) = 0")
problem.add_bc("left(w) = 0")
problem.add_bc("right(w) = 0")


ep = Eigenproblem(problem)

2020-05-01 17:24:05,536 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


Now, we construct an output dictionary `results`, and use `eigentools` to compute a growth rate for each input parameter set.

In [14]:
results = {μ: {} for μ in KGdP_Re}
for μ in KGdP_Re:
    growth, index, freq = ep.growth_rate(KGdP_Re[μ])
    results[μ]['growth'] = growth
    results[μ]['freq'] = freq[0]


2020-05-01 17:24:08,759 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


/home/joishi/hg-projects/eigentools/eigentools/eigenproblem.py:232: ComplexWarning: Casting complex values to real discards the imaginary part
  indx = lambda1_and_indx[:, 1].astype(np.int)


2020-05-01 17:24:09,886 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2020-05-01 17:24:10,786 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2020-05-01 17:24:11,572 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2020-05-01 17:24:12,367 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2020-05-01 17:24:13,125 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10
2020-05-01 17:24:13,937 problems 0/1 INFO :: Solving EVP with homogeneity tolerance of 1.000e-10


Finally, we check results against the published frequencies. Note that the $\sigma$ in our input table is actually $-\sigma$; we correct for that here.

In [31]:
def check_results(results, reference):
    for μ in results:
        res = -results[μ]['freq']
        ref = reference[μ]['σ']*KGdP_Re[μ]['nu']
        freq_err = (res - ref)/ref
        print("μ = {:5.2f}: dedalus: {:15.12e}; reference: {:15.12e}; relative frequency error = {:5.3e} ".format(μ, res, ref, freq_err))

In [32]:
check_results(results, KGdP_Ta)

μ =  0.00: dedalus: 1.791897295476e-15; reference: 0.000000000000e+00; relative frequency error =   inf 
μ = -0.80: dedalus: 5.683123170937e-02; reference: 5.611625262266e-02; relative frequency error = 1.274e-02 
μ = -1.00: dedalus: 7.678678940519e-02; reference: 7.564731625475e-02; relative frequency error = 1.506e-02 
μ = -1.25: dedalus: 9.370611009154e-02; reference: 9.204411239854e-02; relative frequency error = 1.806e-02 
μ = -1.50: dedalus: 1.073477208760e-01; reference: 1.051166187575e-01; relative frequency error = 2.123e-02 
μ = -1.75: dedalus: 1.108497190220e-01; reference: 1.084172243196e-01; relative frequency error = 2.244e-02 
μ = -2.00: dedalus: 1.223553832510e-01; reference: 1.193011261036e-01; relative frequency error = 2.560e-02 


/home/joishi/miniconda3/envs/dedalus-dev/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


All runs agree to better than 2.5%. This relatively large error probably comes from insufficient precision in the reporting of $\alpha$, the critical $z$ wavenumber reported in the original table.